## Linear Regression Using the Ames Housing Data

Using the Ames Housing Data:

Dean De Cock
Truman State University
Journal of Statistics Education Volume 19, Number 3(2011), www.amstat.org/publications/jse/v19n3/decock.pdf



In this notebook, we will build some linear regression models to predict housing prices from this data.  We will split our data into training and test sets, build various models on the training data and compare their results on the test set. We will examine metrics such as *mean squared error* and *mean absolute deviation*.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
#import ml_insights as mli

# To Plot matplotlib figures inline on the notebook
%matplotlib inline

from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split

from sklearn.linear_model import LinearRegression, Lasso, LassoCV

## Load the Data, Examine and Explore

In [2]:
## Load in the Ames Housing Data
datafile = "../../week02-luther/03-regression_scrape/data/Ames_Housing_Data.tsv"
df=pd.read_csv(datafile, sep='\t')

In [3]:
## Examine the columns, look at missing data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
Order              2930 non-null int64
PID                2930 non-null int64
MS SubClass        2930 non-null int64
MS Zoning          2930 non-null object
Lot Frontage       2440 non-null float64
Lot Area           2930 non-null int64
Street             2930 non-null object
Alley              198 non-null object
Lot Shape          2930 non-null object
Land Contour       2930 non-null object
Utilities          2930 non-null object
Lot Config         2930 non-null object
Land Slope         2930 non-null object
Neighborhood       2930 non-null object
Condition 1        2930 non-null object
Condition 2        2930 non-null object
Bldg Type          2930 non-null object
House Style        2930 non-null object
Overall Qual       2930 non-null int64
Overall Cond       2930 non-null int64
Year Built         2930 non-null int64
Year Remod/Add     2930 non-null int64
Roof Style         29

In [4]:
# This is recommended by the data set author to remove a few outliers

df = df.loc[df['Gr Liv Area']<=4000,:]
df.shape

(2925, 82)

There are a *lot* of variables, many of which have a lot of missing values.  Let's pick out just a few columns and start building models using that.

In [5]:
smaller_df= df.loc[:,['Lot Area','Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add',
        'Gr Liv Area', 
        'Full Bath', 'Bedroom AbvGr',
        'Fireplaces', 'Garage Cars','SalePrice']]

In [6]:
smaller_df.describe()

,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Gr Liv Area,Full Bath,Bedroom AbvGr,Fireplaces,Garage Cars,SalePrice
count,2925.000000,2925.000000,2925.000000,2925.000000,2925.000000,2925.000000,2925.000000,2925.000000,2925.000000,2924.000000,2925.000000
mean,10103.583590,6.088205,5.563761,1971.302906,1984.234188,1493.978803,1.564786,2.853675,0.596923,1.765048,180411.574701
std,7781.999124,1.402953,1.112262,30.242474,20.861774,486.273646,0.551386,0.827737,0.645349,0.759834,78554.857286
min,1300.000000,1.000000,1.000000,1872.000000,1950.000000,334.000000,0.000000,0.000000,0.000000,0.000000,12789.000000
25%,7438.000000,5.000000,5.000000,1954.000000,1965.000000,1126.000000,1.000000,2.000000,0.000000,1.000000,129500.000000
50%,9428.000000,6.000000,5.000000,1973.000000,1993.000000,1441.000000,2.000000,3.000000,1.000000,2.000000,160000.000000
75%,11515.000000,7.000000,6.000000,2001.000000,2004.000000,1740.000000,2.000000,3.000000,1.000000,2.000000,213500.000000
max,215245.000000,10.000000,9.000000,2010.000000,2010.000000,3820.000000,4.000000,8.000000,4.000000,5.000000,625000.000000


In [7]:
smaller_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2925 entries, 0 to 2929
Data columns (total 11 columns):
Lot Area          2925 non-null int64
Overall Qual      2925 non-null int64
Overall Cond      2925 non-null int64
Year Built        2925 non-null int64
Year Remod/Add    2925 non-null int64
Gr Liv Area       2925 non-null int64
Full Bath         2925 non-null int64
Bedroom AbvGr     2925 non-null int64
Fireplaces        2925 non-null int64
Garage Cars       2924 non-null float64
SalePrice         2925 non-null int64
dtypes: float64(1), int64(10)
memory usage: 274.2 KB


In [8]:
# There appears to be one NA in Garage Cars - fill with 0
smaller_df = smaller_df.fillna(0)

In [ ]:
smaller_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2925 entries, 0 to 2929
Data columns (total 11 columns):
Lot Area          2925 non-null int64
Overall Qual      2925 non-null int64
Overall Cond      2925 non-null int64
Year Built        2925 non-null int64
Year Remod/Add    2925 non-null int64
Gr Liv Area       2925 non-null int64
Full Bath         2925 non-null int64
Bedroom AbvGr     2925 non-null int64
Fireplaces        2925 non-null int64
Garage Cars       2925 non-null float64
SalePrice         2925 non-null int64
dtypes: float64(1), int64(10)
memory usage: 274.2 KB


In [ ]:
sns.pairplot(smaller_df)

### Data Exploration Questions
1. Which variables seem to have strong relationships with Sales Price?
1. The scatterplots of Year Built vs Year Add/Remod have an interesting structure.  Can you explain what is going on there?
1. In the plot of "Lot Area" vs. "SalePrice", some outliers are making the plot less visually useful.  How can we make the plot look better?

In [ ]:
plt.scatter(smaller_df.loc[smaller_df['Lot Area']<50000,'Lot Area'], smaller_df.loc[smaller_df['Lot Area']<50000,'SalePrice'], alpha=.2)

In [ ]:
#Separate our features from our target

X=smaller_df.loc[:,['Lot Area','Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add',
        'Gr Liv Area', 
        'Full Bath', 'Bedroom AbvGr',
        'Fireplaces', 'Garage Cars']]

y=smaller_df['SalePrice']

In [ ]:
X.info()

In [ ]:
#Split the data 70-30 train/test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)

In [ ]:
X_train.columns

## One Variable Linear Regression
To begin, we will do a simple one variable linear regression, predicting the Sales Price using the Square Footage (Gr Liv Area) of the house.

In [ ]:
# First let us fit only on Living Area (sqft)
selected_columns_1 = ['Gr Liv Area']

In [ ]:
lr_model1 = LinearRegression()
lr_model1.fit(X_train.loc[:,selected_columns_1],y_train)

In [ ]:
lr_model1.coef_, lr_model1.intercept_

### Comprehension Question
What do the coefficients above represent?  How can they be interpreted?

In [ ]:
plt.scatter(X_train['Gr Liv Area'],y_train,alpha=.05)
vec1 = np.linspace(0,4000,1000)
plt.plot(vec1, lr_model1.intercept_ + lr_model1.coef_[0]*vec1)

In [ ]:
### Get the predictions on the training set
train_set_pred1 = lr_model1.predict(X_train.loc[:,selected_columns_1])

In [ ]:
### Get the predictions on the test set
test_set_pred1 = lr_model1.predict(X_test.loc[:,selected_columns_1])

In [ ]:
### Plot the regression line on top of the data

plt.scatter(X_test['Gr Liv Area'],y_test,alpha=.1)
vec1 = np.linspace(0,4000,1000)
plt.plot(vec1, lr_model1.intercept_ + lr_model1.coef_[0]*vec1)

In [ ]:
## Plot predicted vs actual 

plt.scatter(test_set_pred1,y_test,alpha=.1)
plt.plot(np.linspace(0,600000,1000),np.linspace(0,600000,1000))

In [ ]:
## Residual Plot
## Plot predicted vs actual 

plt.scatter(y_test,y_test-test_set_pred1,alpha=.1)
plt.plot(np.linspace(0,600000,1000),np.linspace(0,0,1000))

In [ ]:
# How good is our model on the test set?

# Root Mean Square Error
np.sqrt(np.mean((test_set_pred1 - y_test)**2))

In [ ]:
# Mean Absolute Deviation
(np.mean(np.abs(test_set_pred1 - y_test)))

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
np.sqrt(mean_squared_error(y_test, test_set_pred1)), mean_absolute_error(y_test,test_set_pred1)

### Multiple Linear Regression
We will now do a regression on several variables.  We will no longer be able to see the regression line so simply in a graph, but we can still look at the predicted vs actual and residual plots

In [ ]:
selected_columns_2 = ['Lot Area', 'Overall Qual', 'Year Built', 'Gr Liv Area']

In [ ]:
lr_model2 = LinearRegression()
lr_model2.fit(X_train.loc[:,selected_columns_2],y_train)

In [ ]:
lr_model2.coef_

In [ ]:
lr_model2.intercept_

In [ ]:
list(zip(selected_columns_2,lr_model2.coef_))

### Comprehension Question
What do these coefficients mean?  Why does `'Gr Liv Area'` have a different coefficient this time?

In [ ]:
X.corr()

In [ ]:
test_set_pred2 = lr_model2.predict(X_test.loc[:,selected_columns_2])

In [ ]:
plt.scatter(test_set_pred2,y_test,alpha=.1)
plt.plot(np.linspace(0,600000,1000),np.linspace(0,600000,1000))

In [ ]:
plt.scatter(y_test, y_test-test_set_pred2,alpha=.1)
plt.plot(np.linspace(0,600000,1000),np.linspace(0,0,1000))

In [ ]:
#RMSE
np.sqrt(np.mean((test_set_pred2 - y_test)**2))

In [ ]:
#MAD
(np.mean(np.abs(test_set_pred2 - y_test)))

Next, let us try using all of the variables (in the reduced selection)

In [ ]:
lr_model3 = LinearRegression()
lr_model3.fit(X_train,y_train)

In [ ]:
list(zip(X_train.columns,lr_model3.coef_))

In [ ]:
test_set_pred3 = lr_model3.predict(X_test)

In [ ]:
plt.scatter(test_set_pred3,y_test,alpha=.1)
plt.plot(np.linspace(0,600000,1000),np.linspace(0,600000,1000))

In [ ]:
plt.scatter(y_test,y_test-test_set_pred3,alpha=.1)
plt.plot(np.linspace(0,600000,1000),np.linspace(0,0,1000))

In [ ]:
#RMSE
np.sqrt(np.mean((test_set_pred3 - y_test)**2))

In [ ]:
#MAD
(np.mean(np.abs(test_set_pred3 - y_test)))

### Adding a quadratic factor
Again, we see that our residual plot indicates some non-linearity.

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,4))

ax[0].scatter(X_train['Overall Qual'],y_train, alpha = .05)

ax[1].scatter(X_train['Gr Liv Area'],y_train, alpha = .05)

Let's try adding in `'Overall Qual'` *squared* as a predictor variable

In [ ]:
X['OQ2'] = X['Overall Qual']**2
X.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)

In [ ]:
lr_model4 = LinearRegression()
lr_model4.fit(X_train,y_train)

In [ ]:
list(zip(X_train.columns,lr_model4.coef_))

In [ ]:
test_set_pred4 = lr_model4.predict(X_test)

In [ ]:
plt.scatter(test_set_pred4,y_test,alpha=.1)
plt.plot(np.linspace(0,600000,1000),np.linspace(0,600000,1000))

In [ ]:
plt.scatter(y_test,y_test-test_set_pred4,alpha=.1)
plt.plot(np.linspace(0,600000,1000),np.linspace(0,0,1000))

In [ ]:
#RMSE
np.sqrt(np.mean((test_set_pred4 - y_test)**2))

In [ ]:
#MAD
(np.mean(np.abs(test_set_pred4 - y_test)))

### Day 2

You may have heard of the term *R-squared*.  R-squared measures the "percentage of variance explained by the predictors".  Another way to express this is to calculate the "sum of squares" in two different ways:
1. Using the squared differences between our prediction and the actual value
2. Using the squared differences between the mean of y and the actual value

(So in the second variant above, you can think of it as a really naive prediction where you just guess the mean of y for every entry, regardless of the value of the predictors)

The first number is called the SSE (sum of squares error).  The second number is called the SST (sum of squares total).  So then SSE/SST is what percentage of the total variance *remains* after the predictors are factored in.  Therefore $1-\frac{SSE}{SST}$ is the *explained variance*.  $R^2 = 1-\frac{SSE}{SST}$.

In [ ]:
## R-squared
SSE = np.sum(np.mean((test_set_pred4 - y_test)**2))
SST = np.sum(np.mean((np.mean(y_test) - y_test)**2))
r_squared = 1-SSE/SST
r_squared

In [ ]:
## can also use the sklearn function

from sklearn.metrics import r2_score
r2_score(y_test, test_set_pred4)

In [ ]:
## Each model in sklearn has a designated "score".
## For LinearRegression, r^2 is the designated "score"
## This function does the predictions on X_test, and then computes the R^2 in one step
lr_model4.score(X_test,y_test)

### Regularization
We have been playing around with adding in variables (or transformations of variables), and then seeing if they improve the model or not.  However, this can be a tedious process.

Regularized Linear Regression (sometimes called Penalized Linear Regression) tries to circumvent this by changing the *cost function*.  In "vanilla" linear regression, the coefficients are chosen purely to minimize the Sum of Squared Errors.  In Regularized Linear Regression, there is an additional component of the cost function that penalizes the "size" of the coefficients.  

Why penalize a coefficient?  At the simplest level, it forces a variable to be "worth it" in order to have a coefficient greater than zero.  This intuition extends to the size of the coefficient - in some ways it is a "simpler model" to have smaller coefficients (in absolute value) than larger ones.

Regularized Linear Regression introduces a "nuisance parameter" that says how strongly we want to penalize the coefficients.  At one extreme there is no penalty, and we revert back to "vanilla" Linear Regression.  At the other extreme, the penalty is so onerous that we set all of the coefficients to zero.  In between these two extremes are continuous set of models.  We will discuss how to choose the best value later on when we discuss *cross-validation*.

There are two main "flavors" of Regularized Linear Regression.  In the LASSO, we penalize the sum of the absolute values of the coefficients and in Ridge Regression we penalize the sum of the squares of the coefficients.  LASSO is often preferred for some detailed reasons we will discuss later.

Let's see some examples.

In [ ]:
from sklearn.linear_model import Lasso, Ridge, LassoCV, RidgeCV

In [ ]:
selected_columns_3 = ['Lot Area', 'Overall Qual', 'Overall Cond', 'Year Built','Year Remod/Add', 'Gr Liv Area', 'Full Bath', 'Bedroom AbvGr',
       'Fireplaces', 'Garage Cars', 'OQ2']
#selected_columns_3 = ['Overall Qual','OQ2','Fireplaces','Full Bath','Fireplaces']

#selected_columns_3 = X_train.columns

In [ ]:

lr_model5 = Lasso(alpha = 1000000)
lr_model5.fit(X_train.loc[:,selected_columns_3],y_train)

In [ ]:
list(zip(selected_columns_3,lr_model5.coef_))

Notice how LASSO sets many variables to 0 (with a high enough alpha parameter). This is its ** feature selection ** property. To help remember that LASSO does this and not ridge, look no further than the name: **Least Absolute Shrinkage and Selection Operator** -- the shrinkage part means making the coefficients smaller by penalizing their size in the cost function, and the selection part is zeroing out coefficients. Remember the double SS! 

To highlight a notable difference between LASSO and ridge, we're going to add an additional column that's Lot Area + noise, so that we have two **highly collinear columns**. Then we'll fit a ridge and LASSO model and compare their coefficients.

In [ ]:
np.random.seed(6)

X_train_collinear = X_train.loc[:,selected_columns_3]
X_train_collinear['Lot Area Clone'] = (X_train_collinear['Lot Area'] + 
                                      2500 * np.random.randn(X_train.shape[0]))

X_train_collinear.corr() #notice .95 correlation b/w Lot Area and its "clone"

As a quick aside, let's understand what happens with p-values when there is a lot of collinearity! We are much less sure about our relationships being meaningful. In this case the model does detect the right variable as having a significant relationship, but this need not be the case in general.

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf 

model = sm.OLS(y_train,sm.add_constant(X_train_collinear))
results = model.fit()

results.summary()

Now let's compare and contrast Ridge vs. Lasso

In [ ]:
lr_model_ridge = Ridge(alpha = 1000000000000)
lr_model_ridge.fit(X_train_collinear,y_train)

list(zip(X_train_collinear.columns,lr_model_ridge.coef_))

**Ridge** smoothed out all of the coefficients, **bringing them closer to 0 but not discarding any of them**. Also, it gave **roughly equal weight to the two highly collinear features**.

In [ ]:
lr_model_lasso = Lasso(alpha = 100000)
lr_model_lasso.fit(X_train_collinear,y_train)

list(zip(X_train_collinear.columns,lr_model_lasso.coef_))

Meanwhile, **Lasso zeroed out most of the coefficients**, and **dropped the noisy collinear clone**, performing feature selection to keep the feature we really wanted. 

What are the Pros/Cons of either behavior?

LASSO:
* Pro: great for trimming features and focusing interpretation on a few key ones
* Con: risk of discarding features that are actually useful

Ridge:
* Pro: great for smoothly handling multicollinearity, very nice when working with sparse features 
* Con: will never fully discard features

As always, you have to validate to choose between the two. If the mapping from features to target truly depends on only a few key features, LASSO should outperform. If instead the target actually depends on many features (even if only a little dependent), Ridge should work better.  

In [ ]:
#Back to the original LASSO model: diagnostics

In [ ]:
test_set_pred5 = lr_model5.predict(X_test.loc[:,selected_columns_3])

In [ ]:
plt.scatter(test_set_pred5,y_test,alpha=.1)
plt.plot(np.linspace(0,600000,1000),np.linspace(0,600000,1000))

In [ ]:
#RMSE
np.sqrt(np.mean((test_set_pred5 - y_test)**2))

### Scaling Parameters
One issue with Regularized Linear Regression is that the "size" of a coefficient may be more reflective of the units or scale of the associated variable.  For example, if a distance is measured in millimeters it will have a smaller coefficient than if it is measured in miles.  For this reason, best practice is to "standardize" the variables prior to running a regularized regression.  Standardizing means adding a constant and then dividing by another constant so that the resulting variable has mean 0 and standard deviation 1.  This ensures that the variables are penalized "fairly" with respect to one another.

We demonstrate how to do this below.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
## This step fits the Standard Scaler to the training data
## Essentially it finds the mean and standard deviation of each variable in the training set

std = StandardScaler()
#std.fit(X_train.values.astype(float))
std.fit(X_train)

In [ ]:
## This step applies the scaler to the train set.
## It subtracts the mean it learned in the previous step and then divides by the standard deviation

X_tr = std.transform(X_train)

In [ ]:
## Apply the scaler to the test set

X_te = std.transform(X_test)

In [ ]:
## Note that even though we put a Pandas Dataframe into the scalar, what comes out is a numpy array
## In general, sklearn works on numpy.  It will accept pandas objects by trying to coerce them to numpy arrays
## But it will not output any pandas objects

type(X_train),type(X_tr)

In [ ]:
## Here we can plot histograms of the transformed variables
## Note that they seem to have means of 0 and stddevs of 1
## (though they do not necessarily seem to be normally distributed)

plt.hist(X_tr[:,3])

Now that we have appropriately scaled our variables, we can apply the LASSO as before.

What we did before was technically not good practice since the variables were on different scales.  Certain variables would be (unfairly) penalized more than others.

In [ ]:
## Fit a LASSO model on the standardized data

lr_model7 = Lasso(alpha = 10000)
lr_model7.fit(X_tr,y_train)

In [ ]:
## Note, it is now difficult to interpret the coefficients
## Would have to do the math to translate back to the original scaling

list(zip(X_train.columns,lr_model7.coef_))

## Finding the "best" value of lambda (alpha) with a single train/test split
Here we will first decide on a vector of "candidate" alpha (lambda) values.  Then, for each candidate value, we run the following steps:
1. Standardize the training data
2. Fit a LASSO model on the training data
3. Using the newly trained model, make predictions on both the training data and the test data
4. Find the sum of squares error on both the training set and test set

Then we plot how the errors change for the different values of alpha.

We can then choose the alpha which gives us the best results on the test set.

In [ ]:
alphalist = 10**(np.linspace(-3,4,200))
err_vec_test = np.zeros(len(alphalist))
err_vec_train = np.zeros(len(alphalist))

for i,curr_alpha in enumerate(alphalist):

    steps = [('standardize', StandardScaler()), ('lasso', Lasso(alpha = curr_alpha))]
#    steps = [('standardize', StandardScaler()), ('ridge', Ridge(alpha = curr_alpha))]

    pipe = Pipeline(steps)
    pipe.fit(X_train.loc[:,selected_columns_3], y_train)
    test_set_pred7 = pipe.predict(X_test.loc[:,selected_columns_3])
    err_vec_test[i] = np.sqrt(np.mean((test_set_pred7 - y_test)**2))

    train_set_pred7 = pipe.predict(X_train.loc[:,selected_columns_3])
    err_vec_train[i] = np.sqrt(np.mean((train_set_pred7 - y_train)**2))

In [ ]:
#plot the curves of both the training error and test error as alpha changes

plt.plot(np.log10(alphalist),err_vec_test)
plt.plot(np.log10(alphalist),err_vec_train)

In [ ]:
## This is the minimum error achieved on the test set across the different alpha values we tried

np.min(err_vec_test)

In [ ]:
## This is the value of alpha that gave us the lowest error
alphalist[np.argmin(err_vec_test)]

### Using LassoCV to find the best alpha via Cross-Validation
In the previous, we found the best alpha value by comparing the performance on a single train/test split.  An even better, though more computationally intensive method, is to do a full cross-validation when comparing the different alphas.  Fortunately, the `LassoCV` in sklearn handles this "under the hood".  You pass the `LassoCV` the list of alphas and the number of folds to use for Cross-Validation.  It will do the following:

- For each value of alpha
1. Do a cross-validation and score the result
- Find the value of alpha that gave the best score
- Fit the model on all the data using the value of alpha it just found

Then you can use the `predict` method of the model just as with all of our previous models

In [ ]:
## Scale the data as before
std = StandardScaler()
std.fit(X_train)

In [ ]:
## Scale the Predictors on both the train and test set
X_tr = std.transform(X_train)
X_te = std.transform(X_test)

In [ ]:
# Run the cross validation, find the best alpha, refit the model on all the data with that alpha

alphavec = 10**np.linspace(-3,9,27)

lr_model8 = LassoCV(alphas = alphavec, cv=5)
lr_model8.fit(X_tr,y_train)

In [ ]:
# This is the best alpha value it found
lr_model8.alpha_

In [ ]:
# These are the coefficients when it refit using that best alpha
list(zip(X_train.columns,lr_model8.coef_))

In [ ]:
# Make predictions on the test set using the new model
test_set_pred8 = lr_model8.predict(X_te)

In [ ]:
# Find the RMSE on the test set using that model
np.sqrt(np.mean((test_set_pred8 - y_test)**2))

## LARS_Path
This is a tool used to visualize *all* of the models across the range of different alpha values.  At the far left is the value of alpha where the penalty on coefficients is *so* onerous, that it just sets all of the coefficients to zero.  At the far left is when there is no penalty, and corresponds to the values of the coefficients that you would get from a "vanilla" linear regression.

So each vertical slice corresponds to the coefficients you would get at a particular setting of alpha.  The black dotted lines indicate where a new variable "enters" the model (that is, its coefficient changes from 0 to non-zero).

This is a good way to see which variables are most influential and how their strengths change as you change the value of alpha.

In [ ]:
from sklearn.linear_model import lars_path

In [ ]:
## Scale the variables
std = StandardScaler()
#std.fit(X_train.values.astype(float))
std.fit(X_train)

In [ ]:
X_tr = std.transform(X_train)


In [ ]:
## Note: lars_path takes numpy matrices, not pandas dataframes

print("Computing regularization path using the LARS ...")
alphas, _, coefs = lars_path(X_tr, y_train.values, method='lasso', verbose=True)

In [ ]:
xx = np.sum(np.abs(coefs.T), axis=1)
xx /= xx[-1]

plt.figure(figsize=(10,10))
plt.plot(xx, coefs.T)
ymin, ymax = plt.ylim()
plt.vlines(xx, ymin, ymax, linestyle='dashed')
plt.xlabel('|coef| / max|coef|')
plt.ylabel('Coefficients')
plt.title('LASSO Path')
plt.axis('tight')
plt.legend(X_train.columns)
plt.show()